In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from analystcommunity.write_connection_data_warehouse import runQuery, to_sql

In [2]:
df = pd.read_csv('ALL_SAC_tiers.csv')

In [3]:
# Get the current date in the desired format
current_date = datetime.now().strftime('%Y-%m-%d')

# Add the current date as a new column to the dataframe
df['created_at'] = current_date

In [4]:
df.head()

,city,customer_id,tier,Adjust_CLTV,created_at
0,SPO,19337183.0,5,2283.001764,2024-06-03
1,SPO,19355299.0,4,2603.712347,2024-06-03
2,SPO,19360755.0,9,549.100005,2024-06-03
3,SPO,19565513.0,4,2969.202681,2024-06-03
4,SPO,19567777.0,8,780.023343,2024-06-03


In [5]:
df2 = df.loc[:,['created_at','city','customer_id','tier']]
df2.columns = ['created_at', 'region_code', 'customer_id', 'cltv']

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41167 entries, 0 to 41166
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   created_at   41167 non-null  object 
 1   region_code  41167 non-null  object 
 2   customer_id  41167 non-null  float64
 3   cltv         41167 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.3+ MB


In [7]:
def put_products_demand_estimations(CLTV_df: pd.DataFrame, region_code: str, remove_previous_records: bool = False):
    
    if remove_previous_records:
        del_test_sql = f"""

        DELETE FROM
            lnd_ops.customer_cltv
        WHERE
            region_code = '{region_code}'
        """

        runQuery('ops', del_test_sql)
    
    to_sql(vertical='ops', 
            df_params=CLTV_df,
            table='customer_cltv')

In [8]:
for city in ['CWB','SPO','BHZ','VCP']:
    put_products_demand_estimations(df2.loc[df2.region_code == city], city, False)